<h1 style="font-size:2.0rem; color:green;"> Trends and climatological mean computing using ESA CCI data, ESA CCI gap-filled data, LMDZ-ORCHIDEE_35km data, scf_AMIP_Atlas_1958_2017 data, and scf_HighResMIP_Atlas_1950_2014 data  </h1>  

<h1 style="font-size:1.5rem; color:green;"> Load required libraries </h1> 

In [1]:
# To reload external files automatically (ex: utils)
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

import numpy as np
import pandas as pd
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import xarray as xr
from scipy import stats
import pymannkendall as mk  

In [2]:
# For parallelisation 
from dask.distributed import Client
client = Client(n_workers=16, threads_per_worker=2, memory_limit='32GB')
client

C:\Users\Hamid\anaconda3\lib\site-packages\distributed\node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52734 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:52734/status,
Dashboard: http://127.0.0.1:52734/status,Workers: 16
Total threads: 32,Total memory: 476.84 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52735,Workers: 16
Dashboard: http://127.0.0.1:52734/status,Total threads: 32
Started: Just now,Total memory: 476.84 GiB
Comm: tcp://127.0.0.1:52847,Total threads: 2
Dashboard: http://127.0.0.1:52850/status,Memory: 29.80 GiB
Nanny: tcp://127.0.0.1:52741,


<h1 style="font-size:1.5rem; color:green;"> Initialization of the variables  </h1> 

In [16]:
period = slice('1982-01-01', '2014-12-31')
mons= [1,2,3,4,11,12]
months_labels = ["JAN","FEB","MAR","APR","NOV","DEC"]
path_model_data = r'C:\Data\Snow\SCFG\Models\LMDZ-ORCHIDEE\Modified_data\monthly_values_threshold_1'
path_esa_data = r'C:\Data\Snow\SCFG\Satellite\modified_data\monthly_values_threshold_1'
path_era_land = r'C:\Data\Snow\SCFG\Reanalysis\ERA-LAND\monthly_values_threshold_1'

output_path = r'C:\Users\Hamid\Desktop\results\data\spatio-temporal_analysis\Climatologies\trends\trends_threshold_1'              

<h1 style="font-size:1.5rem; color:green;"> Importing and preparing data</h1>

In [4]:
scf_monthly_values_esa_orig = xr.open_dataset(path_esa_data +r'\ESA_CCI_ORIG_variables_monthly_values_threshold_1.nc').sel(time=period).load()
scf_monthly_values_esa_gf = xr.open_dataset(path_esa_data +r'\ESA_CCI_GF_variables_monthly_values_threshold_1.nc').sel(time=period).load()
scf_monthly_values_lmdz_saloua_35km = xr.open_dataset(path_model_data +r'\scf_lmdz_variables_monthly_values_threshold_1.nc').sel(time=period).load()
scf_monthly_values_amip_250km = xr.open_dataset(path_model_data +r'\scf_AMIP_Atlas_1958_2017_variables_monthly_values_threshold_1.nc').sel(time=period).load()
scf_monthly_values_high_res_mip_50km = xr.open_dataset(path_model_data +r'\scf_HighResMIP_Atlas_1950_2014_variables_monthly_values_threshold_1.nc').sel(time=period).load()
scf_monthly_value_era_land_9km = xr.open_dataset(path_era_land + r'\scf_ERA_Land_1950_2022_variables_monthly_values_threshold_1.nc').sel(time=period).load()

In [5]:
keys_dic = ["ESA_ORIG","ESA_GF","SIM_35k","SIM_250km","SIM_50km"]
monthly_values = {"ESA_ORIG":scf_monthly_values_esa_orig,"ESA_GF":scf_monthly_values_esa_gf,"SIM_35k":scf_monthly_values_lmdz_saloua_35km,"SIM_250km":scf_monthly_values_amip_250km,"SIM_50km":scf_monthly_values_high_res_mip_50km}

<div class="alert alert-block alert-success"; background-color:red> There are 5 missing dates in the time dimension of the
     data scf_monthly_values_esa_orig, we will add these dates and assign them nan values </div> 

In [6]:
"""
We detect missing dates by applying the difference function between a complete dataset and the dataset that is not
complete  
"""
missing_dates = set(monthly_values['ESA_GF'].time.values).difference(set(monthly_values['ESA_ORIG'].time.values))
missing_dates = list(missing_dates)   

# Then we convert the list of missing date to warray dataarray 
missing_date_ds = xr.DataArray(missing_dates, dims=["time"], coords=[missing_dates])

# We constract a dataaray of the whole dates period  
full_dates = xr.concat([scf_monthly_values_esa_orig.time, missing_date_ds], dim="time")

# We reindex the original dataset and fill empty values with nan
scf_monthly_values_esa_orig = scf_monthly_values_esa_orig.reindex(time=full_dates, fill_value=np.nan).sortby("time")


 <h1 style="font-size:1.5rem; color:green;"> The monthly climatological averages without any constraint </h1>

In [ ]:
# We define a dictionnary that will contain the five datasets of monthly averages of all above-mentioned variables 
monthly_values_basic_mean = {key:'a' for key in keys_dic}

In [ ]:
# We compute the averages off all variables for all datasets 
for key in keys_dic:
    monthly_values_basic_mean[key] = monthly_values[key].groupby('time.month').mean(dim='time')

In [ ]:
# We store the results      
for key in keys_dic:
    monthly_values_basic_mean[key].to_netcdf(output_path + '/based_monthly_averages/'+key+'_based_average.nc')                               

In [ ]:
scf_monthly_average_era_land_9km = scf_monthly_value_era_land_9km.groupby('time.month').mean(dim='time')

In [ ]:
scf_monthly_average_era_land_9km

In [ ]:
scf_monthly_average_era_land_9km.to_netcdf(output_path + '/based_monthly_averages/scf_monthly_average_era_land_9km.nc') 

<h1 style="font-size:2rem; color:green;"> Trends </h1>

In [7]:
# We create an auxilary liste of variables containing the names of the variables to which we add _p to designate 
# the p-value relative to this variable and _slope to designate the slope of the trend relative to such variable 
trend_variables = []
for var in list(monthly_values[keys_dic[0]].variables.keys())[3:]:
    trend_variables.append(var+'_slope')              
    trend_variables.append(var+'_p') 

In [8]:
# we initialize the dictionary that will contain the trends  
trend_data = {label+'_trend':'a' for label in keys_dic}

In [9]:
# We store in the obove-mentioned dictionnary the empty datasets that will contain the parameters values of trend
index = 1
for key in list(trend_data.keys())[1:5]:
    empty_ds = monthly_values[keys_dic[index]].drop(list(monthly_values[keys_dic[index]].variables.keys())[3:]).groupby('time.month')
    trend_data[key] = empty_ds
    index = index +1

In [10]:
# We initialize a list of parameters names that will be served to acess the data for each parameter
parameters = ['scf_monthly_value', 'days_with_snow', 'ext_days_with_snow', 'scf_monthly_value_15', 'days_with_snow_15', 'ext_days_with_snow_15', ]

<h1 style="font-size:1.5rem; color:green;"> ESA ORIG </h1>

In [14]:
trend_esa_orig = {parameters[i]:pd.DataFrame(columns=['month', 'lat', 'lon', parameters[i]+'_slope', parameters[i]+'_p']) for i in range(6)} 

In [15]:
lats = monthly_values[keys_dic[0]].lat.values
lons = monthly_values[keys_dic[0]].lon.values

i=0
for var in parameters:
    slope = []  
    p = []   
    la = []   
    lo = []   
    mons = []
    slope=[]
    p=[]
    for mon in [1,2,3,4,11,12]:
        for lat in lats:
            for lon in lons:
                try:
                    trend = mk.original_test(monthly_values[keys_dic[0]].groupby('time.month')[mon].loc[{'lat':lat,'lon':lon}][var].values)
                    slope.append(trend.slope)
                    p.append(trend.p)
                    la.append(lat)
                    lo.append(lon)
                    mons.append(mon)   
                
                except ZeroDivisionError:
                    continue 
                    
    trend_esa_orig[var]['month'] = mons
    trend_esa_orig[var][parameters[i]+'_slope'] = slope
    trend_esa_orig[var][parameters[i]+'_p'] = p
    trend_esa_orig[var]['lat'] = la
    trend_esa_orig[var]['lon'] = lo
    
    print('done for parameter: '+var)
        
    i=i+1

done for parameter: scf_monthly_value
done for parameter: days_with_snow
done for parameter: ext_days_with_snow
done for parameter: scf_monthly_value_15
done for parameter: days_with_snow_15
done for parameter: ext_days_with_snow_15


In [19]:
for par in parameters:
    trend_esa_orig[par].set_index(['month','lon','lat']).to_xarray().to_netcdf(output_path+'/esa_orig_'+par+'_trend_threshold_1.nc')

<h1 style="font-size:1.5rem; color:green;"> ESA GF </h1>

In [21]:
trend_esa_gf = {parameters[i]:pd.DataFrame(columns=['month', 'lat', 'lon', parameters[i]+'_slope', parameters[i]+'_p']) for i in range(6)}  
df = pd.DataFrame(columns=['month', 'lat', 'lon', 'slope', 'p'])  

In [22]:
lats = monthly_values[keys_dic[1]].lat.values
lons = monthly_values[keys_dic[1]].lon.values

i=0
for var in parameters:
    slope = []  
    p = []   
    la = []   
    lo = []   
    mons = []
    slope=[]
    p=[]
    for mon in [1,2,3,4,11,12]:
        for lat in lats:
            for lon in lons:
                try:
                    trend = mk.original_test(monthly_values[keys_dic[1]].groupby('time.month')[mon].loc[{'lat':lat,'lon':lon}][var].values)
                    slope.append(trend.slope)
                    p.append(trend.p)
                    la.append(lat)
                    lo.append(lon)
                    mons.append(mon)   
                
                except ZeroDivisionError:
                    continue 
                    
    trend_esa_gf[var]['month'] = mons
    trend_esa_gf[var][parameters[i]+'_slope'] = slope
    trend_esa_gf[var][parameters[i]+'_p'] = p
    trend_esa_gf[var]['lat'] = la
    trend_esa_gf[var]['lon'] = lo
    
    print('done for parameter: '+var)
    i=i+1

done for parameter: scf_monthly_value
done for parameter: days_with_snow
done for parameter: ext_days_with_snow
done for parameter: scf_monthly_value_15
done for parameter: days_with_snow_15
done for parameter: ext_days_with_snow_15


In [23]:
for par in parameters:   
    trend_esa_gf[par].set_index(['month','lon','lat']).to_xarray().to_netcdf(output_path+'/esa_gf_'+par+'_trend.nc')

In [24]:
trend_SIM_35km = {parameters[i]:pd.DataFrame(columns=['month', 'lat', 'lon', parameters[i]+'_slope', parameters[i]+'_p']) for i in range(6)}  
df = pd.DataFrame(columns=['month', 'lat', 'lon', 'slope', 'p'])  

In [25]:
lats = monthly_values[keys_dic[2]].lat.values
lons = monthly_values[keys_dic[2]].lon.values

i=0
for var in parameters:
    slope = []  
    p = []   
    la = []   
    lo = []   
    mons = []
    slope=[]
    p=[]
    for mon in [1,2,3,4,11,12]:
        for lat in lats:
            for lon in lons:
                try:
                    trend = mk.original_test(monthly_values[keys_dic[2]].groupby('time.month')[mon].loc[{'lat':lat,'lon':lon}][var].values)
                    slope.append(trend.slope)
                    p.append(trend.p)
                    la.append(lat)
                    lo.append(lon)
                    mons.append(mon)   
                
                except ZeroDivisionError:
                    continue 
                    
    trend_SIM_35km[var]['month'] = mons
    trend_SIM_35km[var][parameters[i]+'_slope'] = slope
    trend_SIM_35km[var][parameters[i]+'_p'] = p
    trend_SIM_35km[var]['lat'] = la
    trend_SIM_35km[var]['lon'] = lo
    
    print('done for parameter: '+var)
    
    i=i+1

done for parameter: scf_monthly_value
done for parameter: days_with_snow
done for parameter: ext_days_with_snow
done for parameter: scf_monthly_value_15
done for parameter: days_with_snow_15
done for parameter: ext_days_with_snow_15


In [26]:
for par in parameters:
    trend_SIM_35km[par].set_index(['month','lon','lat']).to_xarray().to_netcdf(output_path+'/SIM_35km_'+par+'_trend.nc')

In [27]:
trend_SIM_250km = {parameters[i]:pd.DataFrame(columns=['month', 'lat', 'lon', parameters[i]+'_slope', parameters[i]+'_p']) for i in range(6)}  
df = pd.DataFrame(columns=['month', 'lat', 'lon', 'slope', 'p'])  

In [28]:
lats = monthly_values[keys_dic[3]].lat.values
lons = monthly_values[keys_dic[3]].lon.values

ctr = 0
i=0
for var in parameters:
    slope = []  
    p = []   
    la = []   
    lo = []   
    mons = []
    slope=[]
    p=[]
    for mon in [1,2,3,4,11,12]:
        for lat in lats:
            for lon in lons:
                try:
                    trend = mk.original_test(monthly_values[keys_dic[3]].groupby('time.month')[mon].loc[{'lat':lat,'lon':lon}][var].values)
                    slope.append(trend.slope)
                    p.append(trend.p)
                    la.append(lat)
                    lo.append(lon)
                    mons.append(mon)   
                
                except ZeroDivisionError:
                    continue 
                    
    trend_SIM_250km[var]['month'] = mons
    trend_SIM_250km[var][parameters[i]+'_slope'] = slope
    trend_SIM_250km[var][parameters[i]+'_p'] = p
    trend_SIM_250km[var]['lat'] = la
    trend_SIM_250km[var]['lon'] = lo
    
    print('done for parameter :'+var)
    i=i+1

done for parameter :scf_monthly_value
done for parameter :days_with_snow
done for parameter :ext_days_with_snow
done for parameter :scf_monthly_value_15
done for parameter :days_with_snow_15
done for parameter :ext_days_with_snow_15


In [29]:
for par in parameters:
    trend_SIM_250km[par].set_index(['month','lon','lat']).to_xarray().to_netcdf(output_path+'/SIM_250km_'+par+'_trend.nc')

In [30]:
trend_SIM_50km = {parameters[i]:pd.DataFrame(columns=['month', 'lat', 'lon', parameters[i]+'_slope', parameters[i]+'_p']) for i in range(6)}  
df = pd.DataFrame(columns=['month', 'lat', 'lon', 'slope', 'p']) 

In [31]:
lats = monthly_values[keys_dic[4]].lat.values
lons = monthly_values[keys_dic[4]].lon.values

i=0
for var in parameters:
    slope = []  
    p = []   
    la = []   
    lo = []   
    mons = []
    slope=[]
    p=[]
    for mon in [1,2,3,4,11,12]:
        for lat in lats:
            for lon in lons:
                try:
                    trend = mk.original_test(monthly_values[keys_dic[4]].groupby('time.month')[mon].loc[{'lat':lat,'lon':lon}][var].values)
                    slope.append(trend.slope)
                    p.append(trend.p)
                    la.append(lat)
                    lo.append(lon)
                    mons.append(mon)   
                
                except ZeroDivisionError:
                    continue 
                    
    trend_SIM_50km[var]['month'] = mons
    trend_SIM_50km[var][parameters[i]+'_slope'] = slope
    trend_SIM_50km[var][parameters[i]+'_p'] = p
    trend_SIM_50km[var]['lat'] = la
    trend_SIM_50km[var]['lon'] = lo
    print('done for parameter :'+ var)
    
    i=i+1

done for parameter :scf_monthly_value
done for parameter :days_with_snow
done for parameter :ext_days_with_snow
done for parameter :scf_monthly_value_15
done for parameter :days_with_snow_15
done for parameter :ext_days_with_snow_15


In [36]:
for par in parameters:
    trend_SIM_50km[par].set_index(['month','lon','lat']).to_xarray().to_netcdf(output_path+'/SIM_50km_'+par+'_trend.nc')

In [34]:
trend_era_land = {parameters[i]:pd.DataFrame(columns=['month', 'lat', 'lon', parameters[i]+'_slope', parameters[i]+'_p']) for i in range(6)}  
df = pd.DataFrame(columns=['month', 'lat', 'lon', 'slope', 'p'])  

In [35]:
lats = scf_monthly_value_era_land_9km.lat.values
lons = scf_monthly_value_era_land_9km.lon.values

i=0
for var in parameters:
    slope = []  
    p = []   
    la = []   
    lo = []   
    mons = []
    slope=[]
    p=[]
    for mon in [1,2,3,4,11,12]:
        for lat in lats:
            for lon in lons:
                try:
                    trend = mk.original_test(scf_monthly_value_era_land_9km.groupby('time.month')[mon].loc[{'lat':lat,'lon':lon}][var].values)
                    slope.append(trend.slope)
                    p.append(trend.p)
                    la.append(lat)
                    lo.append(lon)
                    mons.append(mon)   
                
                except ZeroDivisionError:
                    continue 
                    
    trend_era_land[var]['month'] = mons
    trend_era_land[var][parameters[i]+'_slope'] = slope
    trend_era_land[var][parameters[i]+'_p'] = p
    trend_era_land[var]['lat'] = la
    trend_era_land[var]['lon'] = lo
    
    print('done for parameter: '+var)
    
    i=i+1

done for parameter: scf_monthly_value
done for parameter: days_with_snow
done for parameter: ext_days_with_snow
done for parameter: scf_monthly_value_15
done for parameter: days_with_snow_15
done for parameter: ext_days_with_snow_15


In [37]:
for par in parameters:
    trend_era_land[par].set_index(['month','lon','lat']).to_xarray().to_netcdf(output_path+'/ERA_LAND_'+par+'_trend.nc')